In [658]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import collections
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_string_dtype


In [673]:
df2 = pd.read_csv('cup98lrn.txt', sep=',')
df1 = pd.read_csv('cup98lrn.txt', sep=',')
type_dict = pd.read_csv('otype1.txt', header=None, index_col=0, squeeze=True).to_dict()

/tmp/ipykernel_762942/3255863242.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('cup98lrn.txt', sep=',')
/tmp/ipykernel_762942/3255863242.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('cup98lrn.txt', sep=',')
/tmp/ipykernel_762942/3255863242.py:3: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  type_dict = pd.read_csv('otype1.txt', header=None, index_col=0, squeeze=True).to_dict()


In [674]:
'''
Data preprocessing tasks include the following(https://kdd.org/cupfiles/KDDCupData/1998/cup98doc.txt)

1. Noisy Data
Some of the fields in the analysis file may contain data entry and/or
formatting errors. You are expected to clean these fields (without
excluding the records.)
Rui: I think dataframe already takes care of this part. Dont think there is any data record as formatting issue

2. Records and Fields with Missing and Sparse Data
    2.1:Drop the attribute when it missing more than 99.5% 
    2.2:replace the missing record with the (mean,mode,avg, most common type, etc) 
            value of the avg. value of attribute
3.Fields Containing Constants
4.Time Frame and Date Fields     
'''

def split_types(in_dict):
    res = collections.defaultdict(list)
    for key, val in in_dict.items():
        res[val].append(key)
    return res

def clean_whitespace_in_features(df):
    for column in df.columns:
        uniques = df[column].unique()
        if len(uniques) == 2 and ' ' in uniques:
            if is_string_dtype(df[column]):
                df[column] = df[column].replace(to_replace=' ', value='0')
            else:
                df[column] = df[column].replace(to_replace=' ', value=0)    
        
        if len(uniques) > 2 and ' ' in uniques:
            df[column] = df[column].replace(to_replace=' ', value=np.nan)
        
def preprocessing_data(df):    
    ##some features had to manual preprocess#####
    ###truncate the ZIP atrribute to length=5####
    
    type_list = split_types(type_dict)   
#     res = get_wrong_types(df1, type_list)
    
    df['RFA_2F'] = df1['RFA_2F'].astype(str)
    df['ZIP'] = df1['ZIP'].astype(str)
    df['ZIP'] = df1['ZIP'].str.slice(0,5)
    
    
    
    ''' General:
        replacing any value with period or/and whitespace
    '''
    
    
    #whitesapce \s
    replace_string_type_missing_values(type_list['Char'], df)
    replace_numeric_type_missing_values(type_list['Num'], df)
    
    df.drop(labels=['CONTROLN', 'TARGET_B'], axis = 1)
    
    
    ####Fields Containing Constants################
    df.dropna(axis=1, thresh= 2, inplace=True)
    
    ####dealing with missing features#################   
    #1. drop the attribute if missing values >= 99.5%
    #calculating the dropping_treshold 
    num_rows = len(df)
    perc = 99.5
    min_count =  int(((100-perc)/100)*num_rows+ 1)
    df.dropna(axis = 1, thresh=min_count)
    
    #2. if features contains NAN < 99.5% we need to replace NAN with the most frequent value
    #this line does replace differnet attribute types(Number, char, boolean, etc)  with the most frequent
    # value
    df.fillna(df.mode().iloc[0])
    
    
    ####Time Frame and Date Fields#########
    df['temp_end_date'] = 9706


def get_df_data_types(df):
    for column in df.columns:
        print(column, df[column].unique())

def get_wrong_types(df, in_dict):
    res = collections.defaultdict(list)
    for column in df.columns:
        if column in in_dict['Num'] and not is_numeric_dtype(df[column]):
            res['Num'] = column
        
        if column in in_dict['Char'] and not is_string_dtype(df[column]):
            res['Char'] = column
    return res;

def replace_string_type_missing_values(string_types, df):
    for col in string_types:
        counts = len(df[col].unique())
        if counts == 2:
            df[col] = df[col].replace(to_replace=' ', value='Q')
        elif counts > 2:
            df[col] = df[col].replace(to_replace=' ', value=np.nan)         
        
def replace_numeric_type_missing_values(num_types, df):
    for col in num_types:
        counts = len(df[col].unique())
        if counts == 2:
            df[col] = df[col].replace(r'\s.*$', value=9, regex=True)
        elif counts > 2:
            df[col] = df[col].replace(r'\s.*$', value=np.nan, regex=True)
        
def perform_one_hot_encoding(list_attributes, df):
    for attr in list_attributes:
        pd.get_dummies(df[attr], prefix=attr)

    
    

In [676]:
preprocessing_data(df1)
display(df1)


,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,temp_end_date
0,8901,GRI,0,IL,61081,Q,NaN,3712,0,Q,...,0,L,4,E,X,X,X,39.0,C,9706
1,9401,BOA,1,CA,91326,Q,NaN,5202,0,Q,...,0,L,2,G,X,X,X,1.0,A,9706
2,9001,AMH,1,NC,27017,Q,NaN,0,0,Q,...,1,L,4,E,X,X,X,60.0,C,9706
3,8701,BRY,0,CA,95953,Q,NaN,2801,0,Q,...,1,L,4,E,X,X,X,41.0,C,9706
4,8601,NaN,0,FL,33176,Q,NaN,2001,0,X,...,1,L,2,F,X,X,X,26.0,A,9706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,9601,ASE,1,AK,99504,Q,NaN,0,0,Q,...,0,L,1,G,X,X,X,12.0,C,9706
95408,9601,DCD,1,TX,77379,Q,NaN,5001,0,Q,...,1,L,1,F,X,X,X,2.0,A,9706
95409,9501,MBC,1,MI,48910,Q,NaN,3801,0,Q,...,1,L,3,E,X,X,X,34.0,B,9706
95410,8601,PRV,0,CA,91320,Q,NaN,4005,0,X,...,1,L,4,F,X,X,X,11.0,A,9706
